<a href="https://colab.research.google.com/github/fabbio00/GTF-to-Protein-FASTA-Converter/blob/main/GTF_To_Protein_FASTA_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GTF To Protein FASTA converter

In [ ]:
import re

In [ ]:
import pandas as pd

In [ ]:
def format_fasta(header, sequence):
    return header + '\n' + '\n'.join(re.findall('\w{1,80}', sequence))

In [ ]:
def reverse_complement(sequence):
    sequence = sequence.lower()
    sequence = sequence[::-1]
    complement = {'a':'t', 't':'a', 'c':'g', 'g':'c'}
    return ''.join([complement[c] for c in sequence])

In [ ]:
def compose_feature(feature_list, reference_sequence, strand):
    reconstructed_sequence = ''.join(reference_sequence[f[0]-1:f[1]] for f in sorted(feature_list))
    if strand == '-':
        reconstructed_sequence = reverse_complement(reconstructed_sequence)
    return reconstructed_sequence

### Input parameters

In [ ]:
gtf_file_name = './input.gtf'
df = pd.read_csv('./input.gtf', sep = '\t', header = None)
reference_file_name = './ENm006.fa'
genetic_code_name = './genetic-code.txt'
feature_name = 'CDS'

### Reading the genetic code file

In [ ]:
with open(genetic_code_name, 'r') as input_file:
    genetic_code_rows = input_file.readlines()
genetic_code_rows

['A,gct,gcc,gca,gcg\n',
 'C,tgt,tgc\n',
 'D,gat,gac\n',
 'E,gaa,gag\n',
 'F,ttt,ttc\n',
 'G,ggt,ggc,gga,ggg\n',
 'H,cat,cac\n',
 'I,att,atc,ata\n',
 'K,aaa,aag\n',
 'L,tta,ttg,ctt,ctc,cta,ctg\n',
 'M,atg\n',
 'N,aat,aac\n',
 'P,cct,ccc,cca,ccg\n',
 'Q,caa,cag\n',
 'R,cgt,cgc,cga,cgg,aga,agg\n',
 'S,tct,tcc,tca,tcg,agt,agc\n',
 'T,act,acc,aca,acg\n',
 'V,gtt,gtc,gta,gtg\n',
 'W,tgg\n',
 'Y,tat,tac\n',
 's,tga,taa,tag']

### Construction of the genetic code dictionary

In [ ]:
splitted_genetic_code = [row.rstrip().split(',') for row in genetic_code_rows]
tuple_list = [(codon, ammino_list[0]) for ammino_list in splitted_genetic_code for codon in ammino_list[1:]]
genetic_code_dict = dict(tuple_list)
genetic_code_dict

{'gct': 'A',
 'gcc': 'A',
 'gca': 'A',
 'gcg': 'A',
 'tgt': 'C',
 'tgc': 'C',
 'gat': 'D',
 'gac': 'D',
 'gaa': 'E',
 'gag': 'E',
 'ttt': 'F',
 'ttc': 'F',
 'ggt': 'G',
 'ggc': 'G',
 'gga': 'G',
 'ggg': 'G',
 'cat': 'H',
 'cac': 'H',
 'att': 'I',
 'atc': 'I',
 'ata': 'I',
 'aaa': 'K',
 'aag': 'K',
 'tta': 'L',
 'ttg': 'L',
 'ctt': 'L',
 'ctc': 'L',
 'cta': 'L',
 'ctg': 'L',
 'atg': 'M',
 'aat': 'N',
 'aac': 'N',
 'cct': 'P',
 'ccc': 'P',
 'cca': 'P',
 'ccg': 'P',
 'caa': 'Q',
 'cag': 'Q',
 'cgt': 'R',
 'cgc': 'R',
 'cga': 'R',
 'cgg': 'R',
 'aga': 'R',
 'agg': 'R',
 'tct': 'S',
 'tcc': 'S',
 'tca': 'S',
 'tcg': 'S',
 'agt': 'S',
 'agc': 'S',
 'act': 'T',
 'acc': 'T',
 'aca': 'T',
 'acg': 'T',
 'gtt': 'V',
 'gtc': 'V',
 'gta': 'V',
 'gtg': 'V',
 'tgg': 'W',
 'tat': 'Y',
 'tac': 'Y',
 'tga': 's',
 'taa': 's',
 'tag': 's'}

### Reading the `FASTA` file of the reference genomics

In [ ]:
with open(reference_file_name, 'r') as input_file:
    reference_file_rows = input_file.readlines()

In [ ]:
reference_file_rows

['>ENm006\n',
 '...\n',
 ...]

You want to get the reference genomics as a single string

In [ ]:
genomic_reference = ''.join(reference_file_rows[1:]).rstrip().replace('\n', '')
genomic_reference

'...'

### Reading the *records* of the `GTF` file

In [ ]:
with open(gtf_file_name, 'r') as input_file:
    gtf_file_rows = input_file.readlines()

In [ ]:
gtf_file_rows

['ENm006\tVEGA_Known\texon\t71783\t71788\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t70312\t70440\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t69989\t70210\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64935\t65036\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64566\t64673\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52

**Alternative solution**: gtf file treated with pandas. The column is transformed

In [ ]:
name_dict={0: 'reference', 1: 'source', 2: 'feature', 3: 'start', 4: 'end', 5: 'score', 6: 'strand', 7: 'frame', 8: 'attributes'}
df.rename(columns = name_dict, inplace = True)
df

,reference,source,feature,start,end,score,strand,frame,attributes
0,ENm006,VEGA_Known,exon,71783,71788,.,-,.,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
1,ENm006,VEGA_Known,CDS,71783,71788,.,-,0,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
2,ENm006,VEGA_Known,exon,70312,70440,.,-,.,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
3,ENm006,VEGA_Known,CDS,70312,70440,.,-,0,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
4,ENm006,VEGA_Known,exon,69989,70210,.,-,.,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
5,ENm006,VEGA_Known,CDS,69989,70210,.,-,0,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
6,ENm006,VEGA_Known,exon,64935,65036,.,-,.,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
7,ENm006,VEGA_Known,CDS,64935,65036,.,-,0,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
8,ENm006,VEGA_Known,exon,64566,64673,.,-,.,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"
9,ENm006,VEGA_Known,CDS,64566,64673,.,-,0,"transcript_id ""U52112.4-005""; gene_id ""ARHGAP4"";"


### Filtering to reconstruct the CDS

In [ ]:
gtf_file_rows = [row for row in gtf_file_rows if row.rstrip().split()[2] == feature_name]

In [ ]:
gtf_file_rows

['ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64385\t64459\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72761\t72963\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72521\t72683\t.\t-\t1\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72253\t72315\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71872\t71965\t.\t-\t0\ttranscript_id "U52112.4

**Alternative solution**: splitting of the attributes column and filtering for CDS reconstruction

In [ ]:
df['gene'] = ''
df['transcript'] = ''
for (index, record) in df.iterrows():
    transcript_id= re.search('transcript_id\s+(.+?);', record['attributes']).group(1).replace('"', '')
    gene_id = re.search('gene_id\s+(.+?);', record['attributes']).group(1).replace('"', '')
    df.loc[index, 'gene'] = gene_id
    df.loc[index, 'transcript'] = transcript_id
df.drop('attributes', axis = 1, inplace = True)
df.drop(index=df[df['feature'] != feature_name].index, inplace=True)
df

,reference,source,feature,start,end,score,strand,frame,gene,transcript
1,ENm006,VEGA_Known,CDS,71783,71788,.,-,0,ARHGAP4,U52112.4-005
3,ENm006,VEGA_Known,CDS,70312,70440,.,-,0,ARHGAP4,U52112.4-005
5,ENm006,VEGA_Known,CDS,69989,70210,.,-,0,ARHGAP4,U52112.4-005
7,ENm006,VEGA_Known,CDS,64935,65036,.,-,0,ARHGAP4,U52112.4-005
9,ENm006,VEGA_Known,CDS,64566,64673,.,-,0,ARHGAP4,U52112.4-005
11,ENm006,VEGA_Known,CDS,64385,64459,.,-,0,ARHGAP4,U52112.4-005
61,ENm006,VEGA_Known,CDS,72761,72963,.,-,0,ARHGAP4,U52112.4-019
63,ENm006,VEGA_Known,CDS,72521,72683,.,-,1,ARHGAP4,U52112.4-019
65,ENm006,VEGA_Known,CDS,72253,72315,.,-,0,ARHGAP4,U52112.4-019
67,ENm006,VEGA_Known,CDS,71872,71965,.,-,0,ARHGAP4,U52112.4-019


### Construction of the dictionary of strands, annotated gene set and frames

In [ ]:
strand_dict = {}
for row in gtf_file_rows:
    strand = row.rstrip().split('\t')[6]
    hugo_name = re.search('gene_id\s"(\w+)";', row, re.M).group(1)
    strand_dict[hugo_name] = strand

In [ ]:
strand_dict

{'ARHGAP4': '-', 'AVPR2': '+'}

Gene set extraction

In [ ]:
gene_set = set(strand_dict)
gene_set

{'ARHGAP4', 'AVPR2'}

We get the ids and compositions in features

In [ ]:
id_dict = {}
composition_dict = {}
frame_dict = {}
only_frame_dict = {}
for row in gtf_file_rows:
    hugo_name = re.search('gene_id\s"(\w+)";', row, re.M).group(1)
    transcript_id = re.search('transcript_id\s+"([^"]+)";', row, re.M).group(1)
    id_dict_value = id_dict.get(hugo_name,set())
    id_dict_value.add(transcript_id)
    id_dict.update([(hugo_name, id_dict_value)])

    feature_start = row.rstrip().split('\t')[3]
    feature_end = row.rstrip().split('\t')[4]
    composition_dict_value = composition_dict.get(transcript_id, list())
    composition_dict_value.append([int(feature_start), int(feature_end)])
    composition_dict.update([(transcript_id, composition_dict_value)])
    if strand_dict[hugo_name] == '+':
        composition_dict[transcript_id].sort()
        if int(feature_start) == min(composition_dict[transcript_id])[0]:
            frame_dict[transcript_id] = row.rstrip().split('\t')[7]
    else:
        composition_dict[transcript_id].sort(reverse=True)
        if int(feature_start) == max(composition_dict[transcript_id])[0]:
            frame_dict[transcript_id] = row.rstrip().split('\t')[7]

**Dictionary of Transcript IDs**

In [ ]:
id_dict

{'ARHGAP4': {'U52112.4-001',
  'U52112.4-003',
  'U52112.4-005',
  'U52112.4-010',
  'U52112.4-011',
  'U52112.4-017',
  'U52112.4-019',
  'U52112.4-020',
  'U52112.4-024'},
 'AVPR2': {'U52112.2-001', 'U52112.2-003'}}

**Dictionary containing for each transcript its start codon and end codon**

In [ ]:
composition_dict

{'U52112.4-005': [[71783, 71788],
  [70312, 70440],
  [69989, 70210],
  [64935, 65036],
  [64566, 64673],
  [64385, 64459]],
 'U52112.4-019': [[72761, 72963],
  [72521, 72683],
  [72253, 72315],
  [71872, 71965]],
 'U52112.4-017': [[72761, 72963],
  [72521, 72683],
  [72253, 72315],
  [71865, 71965]],
 'U52112.4-010': [[63857, 63942],
  [62286, 62346],
  [61857, 61991],
  [61663, 61768],
  [61328, 61561],
  [61169, 61242],
  [60898, 61081]],
 'U52112.4-020': [[72521, 72560],
  [71783, 71965],
  [70724, 70843],
  [70312, 70440],
  [70097, 70210]],
 'U52112.4-003': [[77293, 77359],
  [72761, 72965],
  [72521, 72683],
  [72253, 72315],
  [71783, 71965],
  [70312, 70440],
  [69989, 70210],
  [64935, 65036],
  [64566, 64757],
  [64375, 64459],
  [64181, 64208],
  [63857, 63959],
  [62286, 62346],
  [62079, 62156],
  [61857, 61991],
  [61663, 61768],
  [61328, 61561],
  [61169, 61242],
  [60898, 61081],
  [60600, 60692],
  [60227, 60326],
  [58886, 59119]],
 'U52112.4-001': [[77293, 77359],


**Dictionary containing frames of each CDS**; for each transcript I have start and end codon with respective frame

In [ ]:
frame_dict

{'U52112.4-005': '0',
 'U52112.4-019': '0',
 'U52112.4-017': '0',
 'U52112.4-010': '2',
 'U52112.4-020': '1',
 'U52112.4-003': '0',
 'U52112.4-001': '0',
 'U52112.4-024': '0',
 'U52112.4-011': '0',
 'U52112.2-003': '0',
 'U52112.2-001': '0'}

**Alternative solution**: dictionary construction

In [ ]:
strand_dict_df = {}
for gene in df['gene'].unique():
    strand_dict_df[gene] = df.loc[df['gene'] == gene, 'strand'].unique()[0]
strand_dict_df

{'ARHGAP4': '-', 'AVPR2': '+'}

In [ ]:
id_dict_df = {}
for gene in df['gene'].unique():
    id_dict_df[gene] = list(df.loc[df['gene'] == gene, 'transcript'].unique())
id_dict_df

{'ARHGAP4': ['U52112.4-005',
  'U52112.4-019',
  'U52112.4-017',
  'U52112.4-010',
  'U52112.4-020',
  'U52112.4-003',
  'U52112.4-001',
  'U52112.4-024',
  'U52112.4-011'],
 'AVPR2': ['U52112.2-003', 'U52112.2-001']}

In [ ]:
composition_dict_df = {}
for transcript in df['transcript'].unique():
    starts = list(df.loc[df['transcript'] == transcript, 'start'])
    ends = list(df.loc[df['transcript'] == transcript, 'end'])
    composition_dict_df[transcript] = [list(element) for element in list(zip(starts,ends))]
composition_dict_df

{'U52112.4-005': [[71783, 71788],
  [70312, 70440],
  [69989, 70210],
  [64935, 65036],
  [64566, 64673],
  [64385, 64459]],
 'U52112.4-019': [[72761, 72963],
  [72521, 72683],
  [72253, 72315],
  [71872, 71965]],
 'U52112.4-017': [[72761, 72963],
  [72521, 72683],
  [72253, 72315],
  [71865, 71965]],
 'U52112.4-010': [[63857, 63942],
  [62286, 62346],
  [61857, 61991],
  [61663, 61768],
  [61328, 61561],
  [61169, 61242],
  [60898, 61081]],
 'U52112.4-020': [[72521, 72560],
  [71783, 71965],
  [70724, 70843],
  [70312, 70440],
  [70097, 70210]],
 'U52112.4-003': [[77293, 77359],
  [72761, 72965],
  [72521, 72683],
  [72253, 72315],
  [71783, 71965],
  [70312, 70440],
  [69989, 70210],
  [64935, 65036],
  [64566, 64757],
  [64375, 64459],
  [64181, 64208],
  [63857, 63959],
  [62286, 62346],
  [62079, 62156],
  [61857, 61991],
  [61663, 61768],
  [61328, 61561],
  [61169, 61242],
  [60898, 61081],
  [60600, 60692],
  [60227, 60326],
  [58886, 59119]],
 'U52112.4-001': [[77293, 77359],


In [ ]:
frame_dict_df = {}
for transcript in df['transcript'].unique():
    frame_dict_df[transcript] = df.loc[df['transcript'] == transcript, 'frame'].values[0]
frame_dict_df

{'U52112.4-005': '0',
 'U52112.4-019': '0',
 'U52112.4-017': '0',
 'U52112.4-010': '2',
 'U52112.4-020': '1',
 'U52112.4-003': '0',
 'U52112.4-001': '0',
 'U52112.4-024': '0',
 'U52112.4-011': '0',
 'U52112.2-003': '0',
 'U52112.2-001': '0'}

Union of the information obtained, CDS reconstruction is carried out without taking into account the frame

In [ ]:
sequence_fasta_list = []

sequence_type = {'exon' : 'transcript', 'CDS' : 'cds'}

for hugo_name in id_dict:
    for transcript_id in id_dict[hugo_name]:
        r_sequence = compose_feature(composition_dict[transcript_id], genomic_reference, strand_dict[hugo_name])
        header = '>' + hugo_name + '; ' + transcript_id + '; ' +' len=' + str(len(r_sequence)) + '; type = ' + sequence_type[feature_name] + '; strand =' + strand_dict[hugo_name]
        sequence_fasta_list.append((header, r_sequence))
sequence_fasta_list

[('>ARHGAP4; U52112.4-003;  len=2841; type = cds; strand =-',
  'atggccgctcacgggaagctgcggcgggagcgggggctgcaggctgagtatgagacgcaagtcaaagagatgcgctggcagctgagcgagcagctgcgctgcctggagctgcagggcgagctgcggcgggagttgctgcaggagctggcagagttcatgcggcgccgcgctgaggtggagctggaatactcccggggcctggaaaagctggccgagcgcttctccagccgtggaggccgcctggggagcagccgggagcaccaaagcttccggaaggagccgtccctcctgtcgcccttgcactgctgggcggtgctgctgcagcacacgcggcagcagagccgggagagcgcggccctgagtgaggtgctggccgggcccctggcccagcgcctgagtcacattgcagaggacgtggggcgcctggtcaagaagagcagggatctggagcagcagctgcaggatgagctcctggaggtggtctcagagctccagacggccaagaagacgtaccaggcatatcacatggagagcgtgaatgccgaggccaagctccgggaggccgagcggcaggaggagaagcgggcaggccggagtgtccccaccaccaccgctggtgccactgaggcagggcccctccgcaagagctccctcaagaagggagggaggctggtggagaagcggcaggccaagttcatggagcacaaactcaagtgcacaaaggcgcgcaacgagtacctgcttagcctggctagtgtcaacgctgctgtcagtaactactacctgcatgacgtcttggacctcatggactgctgtgacacagggttccacctggccctggggcaggtgctccggagctacacggccgctgagagccgcacccaagcctcccaagtgcagggcctgggcagcctggaagaagctgtggaggccct

**CDS reconstruction**; in this case the frame is taken into account by going to change the size of the features according to the frame

In [ ]:
sequence_fasta_list = []
sequence_type = {'exon' : 'transcript', 'CDS' : 'cds'}
bases_left_out_dict = {}
for hugo_name in id_dict:
    for transcript_id in id_dict[hugo_name]:
        bases_left_out_dict[transcript_id] = ' '
        if strand_dict[hugo_name] == '+':
                    #se ho frame 1: elimino il carattere in prima posizione
                    #se ho frame 2: elimino i primi due caratteri
            if frame_dict[transcript_id] == '1':
                composition_dict[transcript_id][0][0] = composition_dict[transcript_id][0][0] + 1
                bases_left_out_dict[transcript_id] = genomic_reference[composition_dict[transcript_id][0][0]]
            if frame_dict[transcript_id] == '2':
                composition_dict[transcript_id][0][0] = composition_dict[transcript_id][0][0] + 2
                bases_left_out_dict[transcript_id] = genomic_reference[composition_dict[transcript_id][0][0]] + genomic_reference[composition_dict[transcript_id][0][0]+1]
        else:
                    #se ho frame 1: elimino l'ultimo carattere della feature
                    #se ho frame 2: elimino gli ultimi 2 car della feature
            if frame_dict[transcript_id] == '1':
                composition_dict[transcript_id][0][1] = composition_dict[transcript_id][0][1] - 1
                bases_left_out_dict[transcript_id] = genomic_reference[composition_dict[transcript_id][0][1]]
            if frame_dict[transcript_id] == '2':
                composition_dict[transcript_id][0][1] = composition_dict[transcript_id][0][1] - 2
                bases_left_out_dict[transcript_id] = genomic_reference[composition_dict[transcript_id][0][1]] + genomic_reference[composition_dict[transcript_id][0][1]-1]
        r_sequence = compose_feature(composition_dict[transcript_id], genomic_reference, strand_dict[hugo_name])
        header = '>' + hugo_name + '; ' + transcript_id + '; ' +' len=' + str(len(r_sequence)) + '; type = ' + sequence_type[feature_name] + '; strand =' + strand_dict[hugo_name] + '; frame = ' + frame_dict[transcript_id] + '; bases_left_out = ' + bases_left_out_dict[transcript_id]
        sequence_fasta_list.append((header, r_sequence))
sequence_fasta_list

[('>ARHGAP4; U52112.4-003;  len=2841; type = cds; strand =-; frame = 0; bases_left_out =  ',
  'atggccgctcacgggaagctgcggcgggagcgggggctgcaggctgagtatgagacgcaagtcaaagagatgcgctggcagctgagcgagcagctgcgctgcctggagctgcagggcgagctgcggcgggagttgctgcaggagctggcagagttcatgcggcgccgcgctgaggtggagctggaatactcccggggcctggaaaagctggccgagcgcttctccagccgtggaggccgcctggggagcagccgggagcaccaaagcttccggaaggagccgtccctcctgtcgcccttgcactgctgggcggtgctgctgcagcacacgcggcagcagagccgggagagcgcggccctgagtgaggtgctggccgggcccctggcccagcgcctgagtcacattgcagaggacgtggggcgcctggtcaagaagagcagggatctggagcagcagctgcaggatgagctcctggaggtggtctcagagctccagacggccaagaagacgtaccaggcatatcacatggagagcgtgaatgccgaggccaagctccgggaggccgagcggcaggaggagaagcgggcaggccggagtgtccccaccaccaccgctggtgccactgaggcagggcccctccgcaagagctccctcaagaagggagggaggctggtggagaagcggcaggccaagttcatggagcacaaactcaagtgcacaaaggcgcgcaacgagtacctgcttagcctggctagtgtcaacgctgctgtcagtaactactacctgcatgacgtcttggacctcatggactgctgtgacacagggttccacctggccctggggcaggtgctccggagctacacggccgctgagagccgcacccaagcctcccaagtgcagggcc

**Alternative solution**: construction of CDS using information obtained with pandas

In [ ]:
sequence_fasta_list_df = []
sequence_type = {'exon' : 'transcript', 'CDS' : 'cds'}
bases_left_out_dict_df = {}
for hugo_name in id_dict_df:
    for transcript_id in id_dict_df[hugo_name]:
        bases_left_out_dict_df[transcript_id] = ' '
        if strand_dict[hugo_name] == '+':
                    #se ho frame 1: elimino il carattere in prima posizione
                    #se ho frame 2: elimino i primi due caratteri
            if frame_dict_df[transcript_id] == '1':
                composition_dict_df[transcript_id][0][0] = composition_dict_df[transcript_id][0][0] + 1
                bases_left_out_dict_df[transcript_id] = genomic_reference[composition_dict_df[transcript_id][0][0]]
            if frame_dict_df[transcript_id] == '2':
                composition_dict_df[transcript_id][0][0] = composition_dict_df[transcript_id][0][0] + 2
                bases_left_out_dict_df[transcript_id] = genomic_reference[composition_dict_df[transcript_id][0][0]] + genomic_reference[composition_dict_df[transcript_id][0][0]+1]
        else:
                    #se ho frame 1: elimino l'ultimo carattere della feature
                    #se ho frame 2: elimino gli ultimi 2 car della feature
            if frame_dict_df[transcript_id] == '1':
                composition_dict_df[transcript_id][0][1] = composition_dict_df[transcript_id][0][1] - 1
                bases_left_out_dict_df[transcript_id] = genomic_reference[composition_dict_df[transcript_id][0][1]]
            if frame_dict_df[transcript_id] == '2':
                composition_dict_df[transcript_id][0][1] = composition_dict_df[transcript_id][0][1] - 2
                bases_left_out_dict_df[transcript_id] = genomic_reference[composition_dict_df[transcript_id][0][1]] + genomic_reference[composition_dict_df[transcript_id][0][1]-1]
        r_sequence = compose_feature(composition_dict_df[transcript_id], genomic_reference, strand_dict_df[hugo_name])
        header = '>' + hugo_name + '; ' + transcript_id + '; ' +' len=' + str(len(r_sequence)) + '; type = ' + sequence_type[feature_name] + '; strand =' + strand_dict_df[hugo_name] + '; frame = ' + frame_dict_df[transcript_id] + '; bases_left_out = ' + bases_left_out_dict_df[transcript_id]
        sequence_fasta_list_df.append((header, r_sequence))
sequence_fasta_list_df

[('>ARHGAP4; U52112.4-005;  len=642; type = cds; strand =-; frame = 0; bases_left_out =  ',
  'gagaagcggcaggccaagttcatggagcacaaactcaagtgcacaaaggcgcgcaacgagtacctgcttagcctggctagtgtcaacgctgctgtcagtaactactacctgcatgacgtcttggacctcatggactgctgtgacacagggttccacctggccctggggcaggtgctccggagctacacggccgctgagagccgcacccaagcctcccaagtgcagggcctgggcagcctggaagaagctgtggaggccctggatcctccaggggacaaagccaaggttctcgaggtgcatgctaccgtcttctgtcccccgctgcgctttgactaccacccccatgatggggatgaggtggctgagatctgcgttgaaatggagctgcgggacgagattctgcccagagcccagaacatccagagccgcctggaccgacagaccattgagacagaggagaccagcccctccaccgagtccctcaagtccaccagctcagacccaggcagccggcaggcgggccggaggcgcggccagcagcaggagaccgaaaccttctacctcacgaagctccaggagtatctgagtggacggagcatcctcgccaagctgcaggccaagcacgagaagctgcaggaggcc'),
 ('>ARHGAP4; U52112.4-019;  len=523; type = cds; strand =-; frame = 0; bases_left_out =  ',
  'atgcgctggcagctgagcgagcagctgcgctgcctggagctgcagggcgagctgcggcgggagttgctgcaggagctggcagagttcatgcggcgccgcgctgaggtggagctggaatactcccggggcctggaaaagctggccgagcgcttctccagccgtgg

**Translation of the CDS into protein** taking into account the changes due to the frame made earlier

In [ ]:
sequence_fasta_list = []

sequence_type = {'exon' : 'transcript', 'CDS' : 'cds'}

for hugo_name in id_dict:
    for transcript_id in id_dict[hugo_name]:
        r_sequence = compose_feature(composition_dict[transcript_id], genomic_reference, strand_dict[hugo_name])
        r_sequence_codon = re.findall('\w{3}', r_sequence.lower(), re.M)
        r_sequence_translate = ''.join([genetic_code_dict[codon] for codon in r_sequence_codon[:-1]])
        header = '>' + hugo_name + '; ' + transcript_id + '; ' +' len=' + str(len(r_sequence_translate)) + '; type = ' + sequence_type[feature_name] + '; strand =' + strand_dict[hugo_name] + '; frame = ' + frame_dict[transcript_id] + '; bases_left_out = ' + bases_left_out_dict[transcript_id]
        sequence_fasta_list.append((header, r_sequence_translate))
sequence_fasta_list

[('>ARHGAP4; U52112.4-003;  len=946; type = cds; strand =-; frame = 0; bases_left_out =  ',
  'MAAHGKLRRERGLQAEYETQVKEMRWQLSEQLRCLELQGELRRELLQELAEFMRRRAEVELEYSRGLEKLAERFSSRGGRLGSSREHQSFRKEPSLLSPLHCWAVLLQHTRQQSRESAALSEVLAGPLAQRLSHIAEDVGRLVKKSRDLEQQLQDELLEVVSELQTAKKTYQAYHMESVNAEAKLREAERQEEKRAGRSVPTTTAGATEAGPLRKSSLKKGGRLVEKRQAKFMEHKLKCTKARNEYLLSLASVNAAVSNYYLHDVLDLMDCCDTGFHLALGQVLRSYTAAESRTQASQVQGLGSLEEAVEALDPPGDKAKVLEVHATVFCPPLRFDYHPHDGDEVAEICVEMELRDEILPRAQNIQSRLDRQTIETEEVNKTLKATLQALLEVVASDDGDVLDSFQTSPSTESLKSTSSDPGSRQAGRRRGQQQETETFYLTKLQEYLSGRSILAKLQAKHEKLQEALQRGDKEEQEVSWTQYTQRKFQKSRQPRPSSQYNQRLFGGDMEKFIQSSGQPVPLVVESCIRFINLNGLQHEGIFRVSGAQLRVSEIRDAFERGEDPLVEGCTAHDLDSVAGVLKLYFRSLEPPLFPPDLFGELLASSELEATAERVEHVSRLLWRLPAPVLVVLRYLFTFLNHLAQYSDENMMDPYNLAVCFGPTLLPVPAGQDPVALQGRVNQLVQTLIVQPDRVFPPLTSLPGPVYEKCMAPPSASCLGDAQLESLGADNEPELEAEMPAQEDDLEGVVEAVACFAYTGRTAQELSFRRGDVLRLHERASSDWWRGEHNGMRGLIPHKYITLPAGTEKQVVGAGLQTAGESGSSPEGLLASELVHRPEPCTSPEAMGPSGHRRRCLVPASPEQHVEVDKAVAQNMDSVFKELLGKTSVRQGLGPASTTSPSPGPR

**Alternative solution**: translation of the CDS

In [ ]:
sequence_fasta_list_df = []

sequence_type = {'exon' : 'transcript', 'CDS' : 'cds'}

for hugo_name in id_dict_df:
    for transcript_id in id_dict_df[hugo_name]:
        r_sequence = compose_feature(composition_dict_df[transcript_id], genomic_reference, strand_dict_df[hugo_name])
        r_sequence_codon = re.findall('\w{3}', r_sequence.lower(), re.M)
        r_sequence_translate = ''.join([genetic_code_dict[codon] for codon in r_sequence_codon[:-1]])
        header = '>' + hugo_name + '; ' + transcript_id + '; ' +' len=' + str(len(r_sequence_translate)) + '; type = ' + sequence_type[feature_name] + '; strand =' + strand_dict_df[hugo_name] + '; frame = ' + frame_dict_df[transcript_id] + '; bases_left_out = ' + bases_left_out_dict_df[transcript_id]
        sequence_fasta_list_df.append((header, r_sequence_translate))
sequence_fasta_list_df

[('>ARHGAP4; U52112.4-005;  len=213; type = cds; strand =-; frame = 0; bases_left_out =  ',
  'EKRQAKFMEHKLKCTKARNEYLLSLASVNAAVSNYYLHDVLDLMDCCDTGFHLALGQVLRSYTAAESRTQASQVQGLGSLEEAVEALDPPGDKAKVLEVHATVFCPPLRFDYHPHDGDEVAEICVEMELRDEILPRAQNIQSRLDRQTIETEETSPSTESLKSTSSDPGSRQAGRRRGQQQETETFYLTKLQEYLSGRSILAKLQAKHEKLQE'),
 ('>ARHGAP4; U52112.4-019;  len=173; type = cds; strand =-; frame = 0; bases_left_out =  ',
  'MRWQLSEQLRCLELQGELRRELLQELAEFMRRRAEVELEYSRGLEKLAERFSSRGGRLGSSREHQSFRKEPSLLSPLHCWAVLLQHTRQQSRESAALSEVLAGPLAQRLSHIAEDVGRLVKKSRDLEQQLQDELLEVVSELQTAKKTYQAYHMESVNAEAKLREAERQEEKRA'),
 ('>ARHGAP4; U52112.4-017;  len=175; type = cds; strand =-; frame = 0; bases_left_out =  ',
  'MRWQLSEQLRCLELQGELRRELLQELAEFMRRRAEVELEYSRGLEKLAERFSSRGGRLGSSREHQSFRKEPSLLSPLHCWAVLLQHTRQQSRESAALSEVLAGPLAQRLSHIAEDVGRLVKKSRDLEQQLQDELLEVVSELQTAKKTYQAYHMESVNAEAKLREAERQEEKRAGR'),
 ('>ARHGAP4; U52112.4-010;  len=291; type = cds; strand =-; frame = 2; bases_left_out = TT',
  'KFQKSRQPRPSSQYNQRLFGGDMEKFIQSSGQPVPLVVESCIRFIN

Verification of the equality of the sequences translated by the two different methods used to repeat and process the data

In [ ]:
sorted(sequence_fasta_list) == sorted(sequence_fasta_list_df)

True

**Getting of FASTA FORMAT**

In [ ]:
sequence_fasta_list = [format_fasta(t[0], t[1]) for t in sequence_fasta_list]

In [ ]:
for seq in sequence_fasta_list:
    print(seq)

>ARHGAP4; U52112.4-003;  len=946; type = cds; strand =-; frame = 0; bases_left_out =  
MAAHGKLRRERGLQAEYETQVKEMRWQLSEQLRCLELQGELRRELLQELAEFMRRRAEVELEYSRGLEKLAERFSSRGGR
LGSSREHQSFRKEPSLLSPLHCWAVLLQHTRQQSRESAALSEVLAGPLAQRLSHIAEDVGRLVKKSRDLEQQLQDELLEV
VSELQTAKKTYQAYHMESVNAEAKLREAERQEEKRAGRSVPTTTAGATEAGPLRKSSLKKGGRLVEKRQAKFMEHKLKCT
KARNEYLLSLASVNAAVSNYYLHDVLDLMDCCDTGFHLALGQVLRSYTAAESRTQASQVQGLGSLEEAVEALDPPGDKAK
VLEVHATVFCPPLRFDYHPHDGDEVAEICVEMELRDEILPRAQNIQSRLDRQTIETEEVNKTLKATLQALLEVVASDDGD
VLDSFQTSPSTESLKSTSSDPGSRQAGRRRGQQQETETFYLTKLQEYLSGRSILAKLQAKHEKLQEALQRGDKEEQEVSW
TQYTQRKFQKSRQPRPSSQYNQRLFGGDMEKFIQSSGQPVPLVVESCIRFINLNGLQHEGIFRVSGAQLRVSEIRDAFER
GEDPLVEGCTAHDLDSVAGVLKLYFRSLEPPLFPPDLFGELLASSELEATAERVEHVSRLLWRLPAPVLVVLRYLFTFLN
HLAQYSDENMMDPYNLAVCFGPTLLPVPAGQDPVALQGRVNQLVQTLIVQPDRVFPPLTSLPGPVYEKCMAPPSASCLGD
AQLESLGADNEPELEAEMPAQEDDLEGVVEAVACFAYTGRTAQELSFRRGDVLRLHERASSDWWRGEHNGMRGLIPHKYI
TLPAGTEKQVVGAGLQTAGESGSSPEGLLASELVHRPEPCTSPEAMGPSGHRRRCLVPASPEQHVEVDKAVAQNMDSVFK
ELLGKTSVRQGLGPASTTSPSP